In [1]:
import pandas as pd
import dill

In [2]:
good = pd.read_csv('good_.tsv', sep='\t')
good.sample(3)

context_2  \
29517                                ладно .   
10152                              поехали .   
11807  зависит где вы установили это , сэр .   

                                 context_1  \
29517                  бобби : интересно ?   
10152                      тебе нравится ?   
11807  сколько вам нужно для всей работы ?   

                                     context_0  \
29517           сейчас станет еще интересней .   
10152  я выиграл ее на " чокнутой лестнице " .   
11807                              50 фунтов .   

                                        reply label  
29517                   быстрей рассказывай .  good  
10152  ты всегда выигрываешь , так нечестно .  good  
11807       вообще - то это довольно дешево .  good

Превращаем тексты в числовые векторы

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
#создаём объект, преобразовывающий короткие фразы в числовые векторы
vectorizer = TfidfVectorizer()
# обучаем его на всех контекствах, запоминаем частоту каждого слова
vectorizer.fit(good.context_0)
# записываем в матрицу сколько раз каждое слово встречалось в тесте
matrix_big = vectorizer.transform(good.context_0)
print(matrix_big.shape) # размер матрицы = число фраз * размер словаря

(59471, 13984)


In [4]:
#импортируем РСА
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300)
#коэффициенты этого преобразования выучиваются так,
# чтобы сохранить максимум информации об исходной матрицы
svd.fit(matrix_big)
matrix_small = svd.transform(matrix_big)
print(matrix_small.shape)
# сохранилось более 40% исходной инфы
print(svd.explained_variance_ratio_.sum())

(59471, 300)
0.44084572534723143


In [5]:
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator

def softmax(x):
    """Функция для создания вероятностного распределения"""
    proba = np.exp(-x)
    return proba / sum(proba)
class NeighborSampler(BaseEstimator):
    """Класс для случайного выбора одного из соседей"""
    def __init__(self, k=5, temperature=1.0):
        self.k = k
        self.temperature = temperature
    def fit(self, X, y):
        self.tree_ = BallTree(X)
        self.y_ = np.array(y)
    def predict(self, X, randome_state=None):
        distances, indices = self.tree_.query(X, return_distance=True, k=self.k)
        result = []
        for distance, index in zip(distances, indices):
            result.append(np.random.choice(index, p=softmax(distance * self.temperature)))
        return self.y_[result]

In [6]:
from sklearn.pipeline import make_pipeline
ns = NeighborSampler()
ns.fit(matrix_small, good.reply)
pipe = make_pipeline(vectorizer, svd, ns)

In [7]:
with open("chat_pipeline.dill", "wb") as f:
    dill.dump(pipe, f)

In [8]:
with open('chat_pipeline.dill', 'rb') as f:
    chat_pipeline = dill.load(f)

In [12]:
print(chat_pipeline.predict(['привет?']))

['о боже , я и не знала , что ты будешь здесь .']
